In [1]:
import dask.array as da
import napari
import btrack
from tqdm.auto import tqdm
from skimage.io import imshow,  imsave, imread
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski
import itertools
import json

plt.rcParams["figure.figsize"] = (15,7)

# Loading images

In [2]:
%%time
image = imread('/mnt/DATA/sandbox/lung_on_chip/monolayer/20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_113.vsi - 005 CF405, CF488, CF561, CF638.tif')


CPU times: user 1.06 s, sys: 1.12 s, total: 2.18 s
Wall time: 2.17 s


### crop image to dispose of blank z slices

In [3]:
image.shape

(81, 2304, 2304, 4)

In [4]:
image = image[25:45]

In [5]:
image.shape

(20, 2304, 2304, 4)

In [6]:
ch0 = image[...,0]
ch1 = image[...,1]
ch2 = image[...,2]
ch3 = image[...,3]

In [7]:
mask_stack = imread('/mnt/DATA/sandbox/lung_on_chip/monolayer/cropped_[25:45]_masks.tif')

In [8]:
mask_stack.shape

(20, 2304, 2304)

### Defining properties to measure similarities of z-slices

Can use these features in latest version of btrack to aid in tracking by comparing features per-frame

In [47]:
# props = ('axis_major_length', 
#          'axis_minor_length', 
#          'eccentricity', 
#          'area', 
#          'mean_intensity', 
#          #'intensity_image', 
#          'orientation')

props = (#'axis_major_length', 
         #'axis_minor_length', 
         #'eccentricity', 
         'area', 
         'mean_intensity',
         #'intensity_image', 
         #'orientation')
)

In [53]:
image.shape, mask_stack.shape

((20, 2304, 2304, 4), (20, 2304, 2304))

In [52]:
objects = btrack.utils.segmentation_to_objects(
    mask_stack, 
    image,#[...,0],
    properties = ('area'))

[INFO][2022/10/31 02:19:33 PM] Localizing objects from segmentation...
[INFO][2022/10/31 02:19:33 PM] Found intensity_image data
[INFO][2022/10/31 02:19:33 PM] Calculating weighted centroids using intensity_image


TypeError: can only concatenate tuple (not "str") to tuple

In [23]:
from skimage.measure import _regionprops

In [24]:
_regionprops.regionprops_table(mask_stack[0], 
                              intensity_image=image[0], 
                              properties=props)

{'area': array([ 382,  800,  607, 1360,  793,  328,  255,  406,  110,   80,  115,
          24,  534,  295,  152,  257,  207,   43, 1073,  144,   39, 1005,
         266,  397,  204,  476,  755,  486,  190,  144,  494,  472,   73,
          92,   22,   47,   28,   86,   66,  109,  277,  522, 1183,  166,
         209, 1071,  513,  997,  998,   20,  397, 1079,  837,  871,  698,
        1800,  184,  370,   50, 1100,  269,   59,   86,  626,  495,   53,
        1268,   91, 1086,   38,  156,  610, 1105,  969,  733,  261, 1154,
         352,   85, 1019,   75,   29,  351,  374,  987,  943,  803,   34,
          62,  388,  550,   66,  178,   18,   24,  467,   71,  243,  483,
         364,   28,  681,   40,  589,  358,  470,  237,  129,  541,   29,
         320, 1207,  422, 1285, 1411, 1111,  142,   32,   35,  860,   72,
          83,  271,   48,   30,  186,   30,  422,  698,   53,  297,  307,
         894,   64,   80,  355,  401,   37,  121,  881,  188, 1045,   93,
         268,  525,  573,  225

In [25]:
_centroids = _regionprops.regionprops_table(mask_stack[0], 
                              intensity_image=image[0], 
                              properties=props)

In [27]:
regions = _regionprops.regionprops(mask_stack[0], intensity_image=image[0],
                          cache=True, extra_properties=None)

In [28]:
regions

In [49]:
properties = list(props)

In [50]:
print(properties)

['area', 'mean_intensity']


In [55]:
output_dict = _regionprops._props_to_dict(regions, properties=properties, separator='-')
_regionprops._props_to_dict(regions, properties=properties, separator='-')

{'area': array([ 382,  800,  607, 1360,  793,  328,  255,  406,  110,   80,  115,
          24,  534,  295,  152,  257,  207,   43, 1073,  144,   39, 1005,
         266,  397,  204,  476,  755,  486,  190,  144,  494,  472,   73,
          92,   22,   47,   28,   86,   66,  109,  277,  522, 1183,  166,
         209, 1071,  513,  997,  998,   20,  397, 1079,  837,  871,  698,
        1800,  184,  370,   50, 1100,  269,   59,   86,  626,  495,   53,
        1268,   91, 1086,   38,  156,  610, 1105,  969,  733,  261, 1154,
         352,   85, 1019,   75,   29,  351,  374,  987,  943,  803,   34,
          62,  388,  550,   66,  178,   18,   24,  467,   71,  243,  483,
         364,   28,  681,   40,  589,  358,  470,  237,  129,  541,   29,
         320, 1207,  422, 1285, 1411, 1111,  142,   32,   35,  860,   72,
          83,  271,   48,   30,  186,   30,  422,  698,   53,  297,  307,
         894,   64,   80,  355,  401,   37,  121,  881,  188, 1045,   93,
         268,  525,  573,  225

### seems as if the problem lies within btrack... as skimage can return a dict formatted table with multichannel intensity. btrack cannot take this table... check that its actually producing the table within the code

In [77]:
objects = btrack.utils.segmentation_to_objects(
    mask_stack, 
    image,
    properties = ('area', 'mean_intensity'))

[INFO][2022/10/31 02:29:01 PM] Localizing objects from segmentation...
[INFO][2022/10/31 02:29:01 PM] Found intensity_image data
[INFO][2022/10/31 02:29:01 PM] Calculating weighted centroids using intensity_image


ValueError: setting an array element with a sequence.

In [80]:
from importlib import reload 
reload(_regionprops)

<module 'skimage.measure._regionprops' from '/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/skimage/measure/_regionprops.py'>

In [82]:
objects = btrack.utils.segmentation_to_objects(
    mask_stack, 
    image[...,0],
    properties = ('area', 'mean_intensity'))

[INFO][2022/10/31 02:36:10 PM] Localizing objects from segmentation...
[INFO][2022/10/31 02:36:10 PM] Found intensity_image data
[INFO][2022/10/31 02:36:10 PM] Calculating weighted centroids using intensity_image


0 centroid_weighted (7.805068578428419, 392.71552503162843)
1 centroid_weighted (9.987648390301096, 423.5506033317592)
2 centroid_weighted (7.261325167037862, 664.8127616926504)
3 centroid_weighted (15.741512713547875, 1372.1669447755264)
4 centroid_weighted (11.826816088057589, 2124.400805599314)
5 centroid_weighted (7.410183669758138, 236.4476086561193)
6 centroid_weighted (8.105249220267169, 370.67677867357145)
7 centroid_weighted (8.297970466283552, 489.9317087364516)
8 centroid_weighted (6.024686781822226, 514.644451034176)
9 centroid_weighted (6.603672842003074, 523.4138014723728)
10 centroid_weighted (9.027031974615573, 537.2775201366853)
11 centroid_weighted (4.470588235294118, 586.3873744619799)
12 centroid_weighted (9.033608582430057, 628.496605755748)
13 centroid_weighted (5.99420104961002, 892.3033430948998)
14 centroid_weighted (4.795337882722387, 936.4206775771484)
15 centroid_weighted (8.850879073756431, 984.94387864494)
16 centroid_weighted (6.779003267973856, 1041.9120

251 centroid_weighted (357.96983589200636, 1937.3631233456856)
252 centroid_weighted (351.84562611012433, 2002.7227131438722)
253 centroid_weighted (362.9111299995278, 596.0158898805307)
254 centroid_weighted (346.38484772212433, 1947.7175937578656)
255 centroid_weighted (351.75130639581954, 2297.2435416206667)
256 centroid_weighted (349.3904761904762, 394.16340852130327)
257 centroid_weighted (358.1146154146037, 1907.6482008498695)
258 centroid_weighted (352.0576571875815, 1039.2334985650925)
259 centroid_weighted (359.919826652221, 1733.6400247639685)
260 centroid_weighted (362.9785327317294, 1854.0917393951695)
261 centroid_weighted (360.58187921405215, 397.07992591949534)
262 centroid_weighted (351.0, 1944.0)
263 centroid_weighted (352.9800995024876, 1946.0)
264 centroid_weighted (353.6100147275405, 1953.118998527246)
265 centroid_weighted (365.7259116250665, 860.5833218415713)
266 centroid_weighted (365.8133053063975, 1970.4686863176137)
267 centroid_weighted (361.1217469338917, 2

236 centroid_weighted (102.00299402815683, 1464.1068176290971)
237 centroid_weighted (93.74957191780823, 1757.6314212328766)
238 centroid_weighted (109.86707776388175, 1237.1800056387308)
239 centroid_weighted (93.6866485013624, 1848.617765667575)
240 centroid_weighted (122.6963682856547, 266.5807227891813)
241 centroid_weighted (107.14250621588728, 435.64520536694266)
242 centroid_weighted (112.62094447258872, 829.7621803484504)
243 centroid_weighted (131.14315083924748, 1114.6139018973593)
244 centroid_weighted (107.90687310508216, 348.8365608904613)
245 centroid_weighted (93.0, 1118.0)
246 centroid_weighted (119.42147102891286, 1371.8489145708097)
247 centroid_weighted (95.5357714895889, 2163.1433529097703)
248 centroid_weighted (111.11672447594631, 2209.3154250238777)
249 centroid_weighted (110.89728650322941, 51.7595052063881)
250 centroid_weighted (111.74728563590111, 1742.6806527775468)
251 centroid_weighted (109.02274324590256, 472.4085256716614)
252 centroid_weighted (101.4177

463 centroid_weighted (215.47102792405764, 49.86873787416852)
464 centroid_weighted (224.99175030388375, 1300.9222655445662)
465 centroid_weighted (199.63580387685292, 1647.2238692512353)
466 centroid_weighted (211.2236827850204, 1910.3241775306055)
467 centroid_weighted (212.20973321902264, 815.6234908467396)
468 centroid_weighted (207.63778450014405, 87.17707433016422)
469 centroid_weighted (218.23990477002485, 247.5632013536823)
470 centroid_weighted (223.61384446720572, 769.5885725756226)
471 centroid_weighted (204.65436800761722, 1180.9948821709118)
472 centroid_weighted (215.69012804318137, 2099.0085066911533)
473 centroid_weighted (202.86923644411655, 2198.0130947989674)
474 centroid_weighted (211.15446936981695, 2240.203271295907)
475 centroid_weighted (230.066014985362, 697.6678658264278)
476 centroid_weighted (216.92326358188427, 1098.8151488358385)
477 centroid_weighted (216.17176283054852, 1606.9856764680767)
478 centroid_weighted (219.96211680953073, 2200.443980338031)
479

...

1685 centroid_weighted (824.0322191897352, 1596.6368340835784)
1686 centroid_weighted (826.8375058003976, 1092.9892527942163)
1687 centroid_weighted (821.3969714019204, 265.70138585568446)
1688 centroid_weighted (835.7071988217604, 180.3542494173597)
1689 centroid_weighted (830.8799646523009, 887.9242988240093)
1690 centroid_weighted (830.7920540904471, 833.4011326782517)
1691 centroid_weighted (833.5827902446678, 1167.3568533758653)
1692 centroid_weighted (834.084368332994, 1694.6398112656357)
1693 centroid_weighted (832.6471214922738, 146.00227278645988)
1694 centroid_weighted (836.8564246248333, 715.570919116411)
1695 centroid_weighted (834.6355817976554, 1759.4304212329207)
1696 centroid_weighted (831.6674632641322, 666.7520002091723)
1697 centroid_weighted (827.54640838345, 1059.1138205611337)
1698 centroid_weighted (839.0230489816856, 1442.6160975474497)
1699 centroid_weighted (846.6357771418711, 1322.1663615360656)
1700 centroid_weighted (832.7080241854705, 1790.8007649019287)
1

1922 centroid_weighted (930.6212363238931, 628.7617309757375)
1923 centroid_weighted (933.5161662170603, 1608.56159465963)
1924 centroid_weighted (930.3478621740273, 2286.4073884722775)
1925 centroid_weighted (930.3260333067284, 399.05845570773124)
1926 centroid_weighted (936.8147135587536, 1688.4686584138926)
1927 centroid_weighted (926.924262224901, 1720.3719321567987)
1928 centroid_weighted (942.7729399097956, 101.79153700231026)
1929 centroid_weighted (935.5332374452827, 141.6898720743532)
1930 centroid_weighted (929.1536512592755, 359.0635851884344)
1931 centroid_weighted (931.5594718804298, 490.44897634687925)
1932 centroid_weighted (926.9471150262672, 1505.4558979016615)
1933 centroid_weighted (927.2829293347886, 1829.0454622393174)
1934 centroid_weighted (937.9838940778194, 1417.9089281984777)
1935 centroid_weighted (927.8766956049918, 196.5464498876056)
1936 centroid_weighted (938.6439369587304, 247.28376783506218)
1937 centroid_weighted (950.9256979011047, 329.64645539261335)

2165 centroid_weighted (1038.831918871452, 838.5716050082149)
2166 centroid_weighted (1047.4503061250614, 1914.769353622201)
2167 centroid_weighted (1047.2118847438985, 1006.5310554865804)
2168 centroid_weighted (1064.3030757542706, 1218.580966278781)
2169 centroid_weighted (1044.353354729092, 1656.9519577119124)
2170 centroid_weighted (1036.695335020421, 279.62106068053964)
2171 centroid_weighted (1042.117271731528, 1960.761715296198)
2172 centroid_weighted (1035.262017491527, 589.9045708990747)
2173 centroid_weighted (1066.4853188437037, 1306.408546601949)
2174 centroid_weighted (1049.2502055973075, 1372.3026645411069)
2175 centroid_weighted (1042.2388723385614, 1156.2537019188644)
2176 centroid_weighted (1055.081979976667, 611.1177188435086)
2177 centroid_weighted (1048.1607263995659, 658.8481436825305)
2178 centroid_weighted (1048.7298904670863, 1056.1239060550888)
2179 centroid_weighted (1060.329753048557, 2167.1948661023252)
2180 centroid_weighted (1045.3334040934742, 313.9824868

2407 centroid_weighted (1131.0883027522937, 580.9178051455923)
2408 centroid_weighted (1143.3939837994058, 902.3681361175561)
2409 centroid_weighted (1147.3169551112203, 1123.7929306311314)
2410 centroid_weighted (1143.555790292405, 1149.5525136112337)
2411 centroid_weighted (1156.0960882044583, 1847.415499372669)
2412 centroid_weighted (1148.3617838534535, 289.36262729161416)
2413 centroid_weighted (1152.4072915367526, 1731.2126877897304)
2414 centroid_weighted (1152.6370234836916, 2291.5952730187055)
2415 centroid_weighted (1149.7712409784879, 1621.4980851826172)
2416 centroid_weighted (1140.887411629443, 2249.12883142934)
2417 centroid_weighted (1143.5892228424216, 4.0934521253756975)
2418 centroid_weighted (1144.9078621834785, 1816.7767704441676)
2419 centroid_weighted (1144.438199136482, 548.8747470419564)
2420 centroid_weighted (1149.7492289226188, 1004.3787699226643)
2421 centroid_weighted (1147.6553543470334, 1443.2916419425637)
2422 centroid_weighted (1140.2685264719662, 2198.

KeyboardInterrupt: 

In [84]:
objects = btrack.utils.segmentation_to_objects(
    mask_stack, 
    image,
    properties = ('area', 'mean_intensity'))

[INFO][2022/10/31 02:36:54 PM] Localizing objects from segmentation...
[INFO][2022/10/31 02:36:54 PM] Found intensity_image data
[INFO][2022/10/31 02:36:54 PM] Calculating weighted centroids using intensity_image


0 centroid_weighted (array([7.80506858, 7.9458953 , 8.12548973, 8.11218683]), array([392.71552503, 392.47479779, 392.67479248, 392.51377837]))
Unexpected exception formatting exception. Falling back to standard exception


TypeError: only size-1 arrays can be converted to Python scalars

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_22444/671153459.py", line 1, in <cell line: 1>
    objects = btrack.utils.segmentation_to_objects(
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/btrack/_localization.py", line 236, in segmentation_to_objects
    _centroids = _centroids_from_single_arr(
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/btrack/_localization.py", line 82, in _centroids_from_single_arr
    _centroids = regionprops_table(
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/skimage/measure/_regionprops.py", line 996, in regionprops_table
  File "/home/dayn/miniconda3/envs/aero/lib/pytho

### it appears to be recording 4 entries for each object, is this the fault of the weighted centroid?

In [107]:
from importlib import reload 
reload(btrack)
reload(btrack._localization)
reload(btrack.utils)
reload(_regionprops)

<module 'skimage.measure._regionprops' from '/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/skimage/measure/_regionprops.py'>

In [105]:
objects = btrack.utils.segmentation_to_objects(
    mask_stack, 
    image,
    properties = ('area', 'mean_intensity'))

[INFO][2022/10/31 02:58:29 PM] Localizing objects from segmentation...
[INFO][2022/10/31 02:58:29 PM] Found intensity_image data
[INFO][2022/10/31 02:58:41 PM] Objects are of type: <class 'dict'>
[INFO][2022/10/31 02:58:41 PM] ...Found 54892 objects in 20 frames.


In [106]:
objects[0]

,ID,x,y,z,t,dummy,states,label,prob,area,mean_intensity-0,mean_intensity-1,mean_intensity-2,mean_intensity-3
0,0,392.458115,8.020942,0.0,0,False,0,5,0.0,382,130.358639,110.363874,174.395288,110.861257


# FIXED

The error lies with the use of weighted centroids. 

In [125]:
from importlib import reload 
reload(btrack._localization)
reload(btrack)
reload(btrack.utils)

<module 'btrack.utils' from '/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/btrack/utils.py'>

In [126]:
objects = btrack.utils.segmentation_to_objects(
    mask_stack, 
    image,
    properties = ('area', 'mean_intensity'), 
    use_weighted_centroid=False)

[INFO][2022/10/31 03:41:35 PM] Localizing objects from segmentation...
[INFO][2022/10/31 03:41:35 PM] Found intensity_image data


4


[INFO][2022/10/31 03:41:47 PM] Objects are of type: <class 'dict'>
[INFO][2022/10/31 03:41:47 PM] ...Found 54892 objects in 20 frames.


In [130]:
from importlib import reload 
reload(btrack._localization)
reload(btrack)
reload(btrack.utils)

<module 'btrack.utils' from '/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/btrack/utils.py'>

In [131]:
objects = btrack.utils.segmentation_to_objects(
    mask_stack, 
    image,
    properties = ('area', 'eccentricity'),#, 'mean_intensity'), 
    use_weighted_centroid=True)

[INFO][2022/10/31 03:43:29 PM] Localizing objects from segmentation...


TypeError: Weighted centroids cannot be used for multichannel images.